# Rustress

Model training

In [0]:
!git clone https://github.com/AIshutin/rustress

fatal: destination path 'rustress' already exists and is not an empty directory.


In [0]:
!unzip rustress/stressdb.zip
!ls

Archive:  rustress/stressdb.zip
replace stressdb.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: rustress  sample_data  stressdb.json


In [0]:
!du -h stressdb.json

111M	stressdb.json


In [0]:
!pip3 install pymorphy2
!pip install -U pymorphy2-dicts-ru

Requirement already up-to-date: pymorphy2-dicts-ru in /usr/local/lib/python3.6/dist-packages (2.4.404381.4453942)


In [0]:
import pymorphy2 as py
morph = py.MorphAnalyzer()
stress_char = "ре́с"[2]

In [0]:
import json
stressdb = json.loads(open('stressdb.json').read())

In [0]:
print(list(stressdb.items())[:10])

[('солнышек', 'со́лнышек'), ('солнышка', 'со́лнышка'), ('солнышках', 'со́лнышках'), ('солнышки', 'со́лнышки'), ('солнышками', 'со́лнышками'), ('солнышку', 'со́лнышку'), ('солнышкам', 'со́лнышкам'), ('солнышке', 'со́лнышке'), ('солнышком', 'со́лнышком'), ('солнышко', 'со́лнышко')]


In [0]:
MAX_VOW = 0
MAX_LENGTH = 0
vow = 'уеыаоэяиюё'
chars = set()
for el in stressdb:
    chars.update(el)
    curr = 0
    for c in el:
        curr += c in vow
    MAX_VOW = max(curr, MAX_VOW)
    MAX_LENGTH = max(MAX_LENGTH, len(el))
print(MAX_LENGTH)
print(chars)

26
{'н', 'Н', 'ш', 'т', 'З', 'ѝ', 'д', 'И', 'ю', 'Ф', 'л', 'Д', 'в', 'м', 'р', 'С', 'г', 'М', 'й', 'А', 'е', 'ц', 'с', 'ї', 'б', 'ж', 'щ', 'ё', 'П', 'Г', 'Е', 'ь', 'ѐ', 'и', 'з', 'ф', 'ъ', 'c', 'Я', 'Э', 'о', 'а', 'В', 'Р', 'э', 'п', 'у', 'К', 'a', 'Б', 'ч', 'e', 'х', 'і', 'к', 'я', 'ы'}


In [0]:
good = '@ёйцукенгшщзхфывапролджэячсмитьъбю'
c2id = dict()
id2c = dict()
for i in range(len(good)):
    id2c[i] = good[i]
    c2id[good[i]] = i

In [0]:
import numpy as np
#import pandas as pd

In [0]:
feat = ['aspect', 'case', 'POS', 'mood', 'gender', 'number', 'person', 'tense', 'voice', 'transitivity']

feat2id = [dict() for i in range(len(feat))]

def enum_feat(key):
  key = key.lower()
  word = morph.parse(key)[0]
  for i in range(len(feat)):
    curr = None
    try:
      curr = eval('word.tag.' + feat[i])
    except:
      pass
    if curr not in feat2id[i]:
      size = len(feat2id)
      feat2id[i][curr] = size

In [0]:
import random as rd
data = list(stressdb.items())
rd.shuffle(data)
X = []
Y = []
cnt1 = 0
cnt2 = 0
cnt3 = 0
s1 = []
s2 = []
for (key, val) in data[:]:
    key = key.lower()
    if 'і' in key:
        cnt1 += 1
        s1.append([key, val])
        continue
    bad = {'ѐ', 'ѐ', 'a', 'c', 'ї'}
    ok = True
    for el in bad:
        if el in key:
            ok = False
            cnt2 += 1
            break
    for el in key:
        if ord(el) < 255:
            ok = False
    if not ok:
        continue
    enum_feat(key)

In [0]:
cnts = [len(feat2id[i]) for i in range(len(feat))]

In [0]:
def prepare_data(key, val):
    key = key.replace('ѝ', 'й')[::-1]
    val = val.replace('ѝ', 'й')[::-1]
    word = morph.parse(key)[0]
    row = np.zeros(MAX_LENGTH * len(chars) + sum(cnts), dtype=np.bool)
    curr = 0
    for i in range(len(feat)):
        #print('word.tag.' + feat[i])
        #print(word.tag.aspect)
        try:
            row[curr + feat2id[eval('word.tag.' + feat[i])]] = 1
        except:
            pass
        curr += cnts[i]
    for i in range(len(key)):
        row[curr + c2id[key[i]] - 1] = 1
        curr += len(chars)
    ind = 0
    for i in range(len(val)):
        if val[i] == stress_char:
            break
        ind += 1
    res = np.zeros(MAX_LENGTH, dtype=np.bool)
    res[ind] = 1
    return (row, res)    

In [0]:
import random as rd
data = list(stressdb.items())
rd.shuffle(data)
X = []
Y = []
cnt1 = 0
cnt2 = 0
cnt3 = 0
s1 = []
s2 = []
for key, val in data[:]:
    key = key.lower()
    val = val.lower()
    if 'і' in key:
        cnt1 += 1
        s1.append([key, val])
        continue
    bad = {'ѐ', 'ѐ', 'a', 'c', 'ї'}
    ok = True
    for el in bad:
        if el in val + key:
            ok = False
            s2.append([el, val, key])
            cnt2 += 1
            break
    for el in val + key:
        if ord(el) < 255:
            ok = False
    if not ok:
        continue
    if key == val:
        cnt3 += 1
        continue
    try:
        (x, y) = prepare_data(key, val)
        X.append(x)
        Y.append(y)
    except Exception as exp:
        print(exp.args, exp)
        print(key, val)
        print(len(key), len(val))
        for el in bad:
            print(el, el in val + key)
        break

In [0]:
prepare_data('вкомпоновываться',  'вкомпоно́вываться')

(array([False, False, False, ..., False, False, False]),
 array([False, False, False, False, False, False, False, False,  True,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False]))

In [0]:
prepare_data('вкомпоновываться',  'вкомпоно́вываться')[0].shape

(1537,)

In [0]:
print(len(X))

847706


In [0]:
print(cnt1, cnt2, cnt3)

18 2635 0


In [0]:
X = np.array(X)
Y = np.array(Y)

In [0]:
np.save('Xstress', X)
np.save('Ystress', Y)

In [0]:
len(X[0])

1492

In [0]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3)

In [0]:
X_train.shape

(593394, 1537)

In [0]:
Y_train.shape

(593394, 26)

In [0]:
ans = [0] 

In [0]:
import keras

Using TensorFlow backend.


In [0]:
from keras.models import Sequential
from keras.layers import Dense

model = Sequential()
model.add(Dense(32, input_dim=X.shape[-1], activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(8, activation='sigmoid'))
model.add(Dense(MAX_LENGTH, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [0]:
model.fit(X_train, Y_train, epochs=30, batch_size=60)

Epoch 1/30
593394/593394 [==============================] - 29s 49us/step - loss: 0.9550 - acc: 0.7034
Epoch 2/30
593394/593394 [==============================] - 28s 47us/step - loss: 0.6277 - acc: 0.7854
Epoch 3/30
593394/593394 [==============================] - 28s 47us/step - loss: 0.5638 - acc: 0.8017
Epoch 4/30
593394/593394 [==============================] - 28s 46us/step - loss: 0.5322 - acc: 0.8102
Epoch 5/30
593394/593394 [==============================] - 28s 47us/step - loss: 0.5123 - acc: 0.8155
Epoch 6/30
593394/593394 [==============================] - 29s 49us/step - loss: 0.4983 - acc: 0.8192
Epoch 7/30
593394/593394 [==============================] - 27s 46us/step - loss: 0.4884 - acc: 0.8218
Epoch 8/30
593394/593394 [==============================] - 28s 46us/step - loss: 0.4806 - acc: 0.8243
Epoch 9/30
593394/593394 [==============================] - 28s 47us/step - loss: 0.4743 - acc: 0.8259
Epoch 10/30
593394/593394 [==============================] - 28s 47us/ste

In [0]:
scores = model.evaluate(X_test, Y_test)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

254312/254312 [==============================] - 7s 28us/step

acc: 83.08%


In [0]:
model.save('model.h5')
with open('c2id.json', 'w', encoding='utf-8') as file:
    print(json.dumps(c2id), file=file)
with open('back.json', 'w') as file:
    print(json.dumps(feat2id), file=file)

TypeError: ignored

In [0]:
!du -h model.h5

620K	model.h5


In [0]:
!ls

adc.json   c2id.json  model.h5	sample_data    Xstress.npy
back.json  drive      rustress	stressdb.json  Ystress.npy


In [0]:
len(chars)

57

In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package libfuse2:amd64.
(Reading database ... 22298 files and directories currently installed.)
Preparing to unpack .../libfuse2_2.9.7-1ubuntu1_amd64.deb ...
Unpacking libfuse2:amd64 (2.9.7-1ubuntu1) ...
Selecting previously unselected package fuse.
Preparing to unpack .../fuse_2.9.7-1ubuntu1_amd64.deb ...
Unpacking fuse (2.9.7-1ubuntu1) ...
Selecting previously unselected package google-drive-ocamlfuse.
Preparing to unpack .../google-drive-ocamlfuse_0.7.1-0ubuntu2~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.1-0ubuntu2~ubuntu18.04.1) ...
Setting up libfuse2:amd64 (2.9.7-1ubuntu1) ...
Processing triggers for libc-bin (2.27-3ubuntu1) ...
Setting up fuse (2.9.7-1ubuntu1) ...
Setting up google-drive-ocamlfuse (0.7.1-0ubuntu2~ubuntu18.04.1) ...


In [0]:
from google.colab import auth
auth.authenticate_user()

In [0]:
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
!mkdir drive/rustress
!cp *.npy drive/rustress/.
!cp model.h5 drive/rustress/model.h5

mkdir: cannot create directory ‘drive/rustress’: File exists


In [0]:
!cp *.json drive/rustress/.